In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Downloading files

In [ ]:
!curl -O "https://raw.githubusercontent.com/SaminRK/scPSS-reproducibility/refs/heads/master/data_downloaders/download_calcagno.sh"
!curl -O "https://raw.githubusercontent.com/SaminRK/scPSS-reproducibility/refs/heads/master/preprocessors/preprocess_calcagno.py"
!curl -O "https://raw.githubusercontent.com/SaminRK/scPSS-reproducibility/refs/heads/master/utils/mean_margin.py"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1445  100  1445    0     0   2956      0 --:--:-- --:--:-- --:--:--  2961
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5339  100  5339    0     0   9358      0 --:--:-- --:--:-- --:--:--  9366
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   987  100   987    0     0   1990      0 --:--:-- --:--:-- --:--:--  1989


In [ ]:
!chmod +x download_calcagno.sh

In [ ]:
!./download_calcagno.sh data

==> Downloading Zenodo ZIP...
--2025-07-21 11:32:53--  https://zenodo.org/records/7055957/files/Nikatag/Single-Cell-Spatial-Transcriptomics-for-Border-zone-BZ_Cell_Mapping.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7972177 (7.6M) [application/octet-stream]
Saving to: ‘/content/data/Single-Cell-Spatial-Transcriptomics-for-Border-zone-BZ_Cell_Mapping_zenodo.zip’

/content/data/Singl 100%[===================>]   7.60M   431KB/s    in 19s     

2025-07-21 11:33:12 (420 KB/s) - ‘/content/data/Single-Cell-Spatial-Transcriptomics-for-Border-zone-BZ_Cell_Mapping_zenodo.zip’ saved [7972177/7972177]

==> Unzipping Zenodo ZIP...
Archive:  /content/data/Single-Cell-Spatial-Transcriptomics-for-Border-zone-BZ_Cell_Mapping_zenodo.zip
8f7e1d14be5fa874c92bff1d8cb282eb9aa613fd
   creating: /content/data/Nikatag-Single-Cel

## scPSS

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install git+https://github.com/SaminRK/scPSS.git

  Cloning https://github.com/SaminRK/scPSS.git to /tmp/pip-req-build-mmftut0s
  Running command git clone --filter=blob:none --quiet https://github.com/SaminRK/scPSS.git /tmp/pip-req-build-mmftut0s
  Resolved https://github.com/SaminRK/scPSS.git to commit c16c71713e6a618ed6196b83b1dc3daa30838e1a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of umap-learn to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 980.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 3.7 MB/s eta 0:00:00
  Created wheel for scPSS: fi

In [ ]:
DATA_DIR = 'data'

In [5]:
## Modify to the directory where outputs should be stored
OUT_PATH = "/content/drive/MyDrive/Research - Sohel sir/Pathological Cells/Pathological Cell Identification/reproducibility/outputs"
metrics_csv_path = f"{OUT_PATH}/scpss-mouse-infarcted-heart-metrics-dist.csv"
results_csv_path = f"{OUT_PATH}/scpss-mouse-infarcted-heart-results-dist.csv"

In [ ]:
from scpss import scPSS
import time
import numpy as np
import traceback
from preprocess_calcagno import get_preprocessed_anndata
import pandas as pd


with open(metrics_csv_path, 'w') as f:
    print('PROBLEM_KEY', 'dist', 'seed', 'accuracy', 'precision', 'recall', 'f1', 'auc', 'aupr', 'outlier_ratio_ref', 'outlier_ratio_que', 'time_taken',  sep=',', file=f, flush=True)

    for PROBLEM_KEY in ['ref_0_que_1hr', 'ref_0_que_4hr', 'ref_0_que_1', 'ref_0_que_3', 'ref_0_que_7',  'ref_0_que_1-7']:
        ad = get_preprocessed_anndata(PROBLEM_KEY, DATA_DIR)

        reference_samples = list(ad.obs[ad.obs['dataset'] == 'reference']['sample'].unique())
        query_samples = list(ad.obs[ad.obs['dataset'] == 'query']['sample'].unique())

        for dist_metric in (
            'mse',
            'mae',
            'braycurtis',
            'canberra',
            'chebyshev',
            'cityblock',
            'cosine',
            'euclidean',
            'mahalanobis',
            'minkowski',
            'seuclidean',
            'sqeuclidean',
        ):
            for i in range(5):
                try:
                    print('RUN: ', i)
                    seed = i * 100

                    start_time = time.perf_counter()

                    scpss = scPSS(ad.copy(), 'sample', reference_samples, query_samples)
                    scpss.harmony_integrate(random_state=seed)
                    scpss.find_optimal_parameters(distance_metric=dist_metric)
                    scpss.set_distance_and_condition()

                    end_time = time.perf_counter()

                    ad_que = scpss.adata[scpss.query_mask]

                    true_labels = ad_que.obs['zone'].isin(['BZ1', 'BZ2']).to_numpy()
                    predicted_labels = ad_que.obs['scpss_condition'].isin(['pathological']).to_numpy()
                    dist_que_ref = scpss.adata.obs.loc[scpss.query_mask, 'scpss_scores']
                    dist_ref_ref = scpss.adata.obs.loc[scpss.reference_mask, 'scpss_scores']

                    time_taken = end_time - start_time


                    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

                    accuracy = accuracy_score(true_labels, predicted_labels)
                    precision = precision_score(true_labels, predicted_labels)
                    recall = recall_score(true_labels, predicted_labels)
                    f1 = f1_score(true_labels, predicted_labels)

                    print("Accuracy:", accuracy)
                    print("Precision:", precision)
                    print("Recall:", recall)
                    print("F1-score:", f1)

                    from sklearn.metrics import roc_auc_score, average_precision_score
                    from sklearn.metrics import roc_curve, precision_recall_curve

                    y_true = true_labels
                    y_scores = dist_que_ref

                    auc = roc_auc_score(y_true, y_scores)
                    print("AUC:", auc)

                    aupr = average_precision_score(y_true, y_scores)
                    print("AUPR:", aupr)

                    outlier_ratio_ref = np.mean(dist_ref_ref > scpss.best_params['threshold'])
                    print("Outlier ratio in reference:", outlier_ratio_ref)
                    outlier_ratio_que = np.mean(dist_que_ref > scpss.best_params['threshold'])
                    print("Outlier ratio in query:", outlier_ratio_que)


                    print(PROBLEM_KEY, dist_metric, seed, accuracy, precision, recall, f1, auc, aupr, outlier_ratio_ref, outlier_ratio_que, time_taken, sep=',', file=f, flush=True)
                except Exception as e:
                    print(e)
                    traceback.print_exc()


RUN:  0


2025-07-21 11:51:54,254 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
INFO:harmonypy:Computing initial centroids with sklearn.KMeans...


✅ PCA Complete.


2025-07-21 11:51:56,464 - harmonypy - INFO - sklearn.KMeans initialization complete.
INFO:harmonypy:sklearn.KMeans initialization complete.
2025-07-21 11:51:56,501 - harmonypy - INFO - Iteration 1 of 10
INFO:harmonypy:Iteration 1 of 10
2025-07-21 11:51:57,857 - harmonypy - INFO - Iteration 2 of 10
INFO:harmonypy:Iteration 2 of 10
2025-07-21 11:51:59,143 - harmonypy - INFO - Converged after 2 iterations
INFO:harmonypy:Converged after 2 iterations


✅ Harmony Integration Complete.
✅ Found Optimal Parameters.
✅ Stored distances and conditions in Anndata object.
Accuracy: 0.7428470505121865
Precision: 0.5987370838117106
Recall: 0.9729477611940298
F1-score: 0.7412935323383084
AUC: 0.9155425614961011
AUPR: 0.8371368612012549
Outlier ratio in reference: 0.03238498789346247
Outlier ratio in query: 0.6153302719886966
RUN:  1


...

✅ Harmony Integration Complete.
✅ Found Optimal Parameters.
✅ Stored distances and conditions in Anndata object.
Accuracy: 0.8184749455337691
Precision: 0.8086001829826166
Recall: 0.8099340175953079
F1-score: 0.809266550682172
AUC: 0.9020276212326654
AUPR: 0.903366877544393
Outlier ratio in reference: 0.032990314769975784
Outlier ratio in query: 0.47625272331154683
RUN:  4


2025-07-21 14:45:18,032 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
INFO:harmonypy:Computing initial centroids with sklearn.KMeans...


✅ PCA Complete.


2025-07-21 14:45:20,819 - harmonypy - INFO - sklearn.KMeans initialization complete.
INFO:harmonypy:sklearn.KMeans initialization complete.
2025-07-21 14:45:20,903 - harmonypy - INFO - Iteration 1 of 10
INFO:harmonypy:Iteration 1 of 10
2025-07-21 14:45:23,984 - harmonypy - INFO - Iteration 2 of 10
INFO:harmonypy:Iteration 2 of 10
2025-07-21 14:45:27,054 - harmonypy - INFO - Converged after 2 iterations
INFO:harmonypy:Converged after 2 iterations


✅ Harmony Integration Complete.
✅ Found Optimal Parameters.
✅ Stored distances and conditions in Anndata object.
Accuracy: 0.8163834422657952
Precision: 0.806068360445988
Recall: 0.8082844574780058
F1-score: 0.8071748878923767
AUC: 0.9008370183081045
AUPR: 0.9023225144677366
Outlier ratio in reference: 0.032990314769975784
Outlier ratio in query: 0.4767755991285403


In [6]:
import pandas as pd
metrics = pd.read_csv(metrics_csv_path)

In [7]:
metrics

,PROBLEM_KEY,dist,seed,accuracy,precision,recall,f1,auc,aupr,outlier_ratio_ref,outlier_ratio_que,time_taken
0,ref_0_que_1hr,mse,0,0.742847,0.598737,0.972948,0.741294,0.915543,0.837137,0.032385,0.615330,45.840185
1,ref_0_que_1hr,mse,100,0.742141,0.598163,0.972015,0.740583,0.914858,0.837088,0.032688,0.615330,45.515187
2,ref_0_que_1hr,mse,200,0.726245,0.583007,0.972948,0.729116,0.917683,0.844002,0.033596,0.631932,44.247327
3,ref_0_que_1hr,mse,300,0.732250,0.588600,0.972948,0.733474,0.919080,0.846998,0.031174,0.625927,45.567256
4,ref_0_que_1hr,mse,400,0.738255,0.594410,0.972015,0.737699,0.915937,0.838947,0.032082,0.619216,43.240942
...,...,...,...,...,...,...,...,...,...,...,...,...
355,ref_0_que_1-7,sqeuclidean,0,0.816645,0.811872,0.799670,0.805725,0.901449,0.901360,0.034201,0.468322,34.998755
356,ref_0_que_1-7,sqeuclidean,100,0.815425,0.802904,0.810850,0.806858,0.901479,0.902908,0.036622,0.480174,35.062088
357,ref_0_que_1-7,sqeuclidean,200,0.816732,0.807220,0.807368,0.807294,0.900762,0.902468,0.031477,0.475556,34.570408
358,ref_0_que_1-7,sqeuclidean,300,0.818475,0.808600,0.809934,0.809267,0.902028,0.903367,0.032990,0.476253,35.205270


In [8]:
import pandas as pd
import numpy as np
from scipy.stats import sem, t

# Confidence level
confidence = 0.95

# Function to calculate mean ± margin
def mean_margin(series, confidence=0.95):
    n = len(series)
    mean = np.mean(series)
    se = sem(series)
    margin = se * t.ppf((1 + confidence) / 2., n - 1)
    return f"{mean:.4f} ± {margin:.4f}"

# Group by both 'PROBLEM_KEY' and 'dist'
grouped = metrics.groupby(['PROBLEM_KEY', 'dist'])

# Apply mean ± margin to all other columns
result = grouped.agg({
    col: lambda x: mean_margin(x, confidence)
    for col in metrics.columns if col not in ['PROBLEM_KEY', 'dist']
})

# Reset index for a cleaner output
result.reset_index(inplace=True)


In [9]:
result

,PROBLEM_KEY,dist,seed,accuracy,precision,recall,f1,auc,aupr,outlier_ratio_ref,outlier_ratio_que,time_taken
0,ref_0_que_1,braycurtis,200.0000 ± 196.3243,0.8353 ± 0.0030,0.7826 ± 0.0038,0.9611 ± 0.0015,0.8627 ± 0.0022,0.9165 ± 0.0038,0.9062 ± 0.0067,0.0333 ± 0.0084,0.6610 ± 0.0037,19.3318 ± 0.5347
1,ref_0_que_1,canberra,200.0000 ± 196.3243,0.8196 ± 0.0090,0.7646 ± 0.0106,0.9606 ± 0.0032,0.8514 ± 0.0062,0.9052 ± 0.0020,0.8696 ± 0.0039,0.0515 ± 0.0040,0.6762 ± 0.0104,21.2853 ± 0.6748
2,ref_0_que_1,chebyshev,200.0000 ± 196.3243,0.8436 ± 0.0035,0.7941 ± 0.0068,0.9578 ± 0.0053,0.8683 ± 0.0020,0.9507 ± 0.0005,0.9591 ± 0.0004,0.0273 ± 0.0032,0.6492 ± 0.0091,18.0558 ± 0.5900
3,ref_0_que_1,cityblock,200.0000 ± 196.3243,0.8460 ± 0.0031,0.7975 ± 0.0047,0.9569 ± 0.0019,0.8700 ± 0.0020,0.9501 ± 0.0006,0.9579 ± 0.0005,0.0262 ± 0.0038,0.6458 ± 0.0050,18.6288 ± 0.6803
4,ref_0_que_1,cosine,200.0000 ± 196.3243,0.8144 ± 0.0023,0.7584 ± 0.0026,0.9614 ± 0.0027,0.8479 ± 0.0017,0.9115 ± 0.0020,0.8986 ± 0.0089,0.0347 ± 0.0008,0.6823 ± 0.0034,17.8522 ± 0.9345
...,...,...,...,...,...,...,...,...,...,...,...,...
67,ref_0_que_7,mahalanobis,200.0000 ± 196.3243,0.6871 ± 0.0111,0.5969 ± 0.0084,0.9346 ± 0.0116,0.7285 ± 0.0090,0.8511 ± 0.0398,0.8148 ± 0.0674,0.0462 ± 0.0080,0.7033 ± 0.0075,49.0162 ± 0.8418
68,ref_0_que_7,minkowski,200.0000 ± 196.3243,0.6390 ± 0.0022,0.5568 ± 0.0015,0.9618 ± 0.0019,0.7053 ± 0.0015,0.8499 ± 0.0076,0.8223 ± 0.0110,0.0347 ± 0.0031,0.7758 ± 0.0022,15.2848 ± 0.5031
69,ref_0_que_7,mse,200.0000 ± 196.3243,0.6390 ± 0.0022,0.5568 ± 0.0015,0.9618 ± 0.0019,0.7053 ± 0.0015,0.8499 ± 0.0076,0.8223 ± 0.0110,0.0347 ± 0.0031,0.7758 ± 0.0022,43.5330 ± 1.0266
70,ref_0_que_7,seuclidean,200.0000 ± 196.3243,0.6737 ± 0.0125,0.5854 ± 0.0094,0.9377 ± 0.0095,0.7208 ± 0.0093,0.8360 ± 0.0252,0.7912 ± 0.0459,0.0455 ± 0.0045,0.7195 ± 0.0087,17.4025 ± 0.3088


In [10]:
result.to_csv(results_csv_path, index=False)

## Compare

In [11]:
## Modify to the directory where outputs should be stored
OUT_PATH = "/content/drive/MyDrive/Research - Sohel sir/Pathological Cells/Pathological Cell Identification/reproducibility/outputs"
metrics_csv_path = f"{OUT_PATH}/scpss-mouse-infarcted-heart-metrics-dist.csv"
results_csv_path = f"{OUT_PATH}/scpss-mouse-infarcted-heart-results-dist.csv"

In [12]:
import pandas as pd
df = pd.read_csv(metrics_csv_path)

In [13]:
df

,PROBLEM_KEY,dist,seed,accuracy,precision,recall,f1,auc,aupr,outlier_ratio_ref,outlier_ratio_que,time_taken
0,ref_0_que_1hr,mse,0,0.742847,0.598737,0.972948,0.741294,0.915543,0.837137,0.032385,0.615330,45.840185
1,ref_0_que_1hr,mse,100,0.742141,0.598163,0.972015,0.740583,0.914858,0.837088,0.032688,0.615330,45.515187
2,ref_0_que_1hr,mse,200,0.726245,0.583007,0.972948,0.729116,0.917683,0.844002,0.033596,0.631932,44.247327
3,ref_0_que_1hr,mse,300,0.732250,0.588600,0.972948,0.733474,0.919080,0.846998,0.031174,0.625927,45.567256
4,ref_0_que_1hr,mse,400,0.738255,0.594410,0.972015,0.737699,0.915937,0.838947,0.032082,0.619216,43.240942
...,...,...,...,...,...,...,...,...,...,...,...,...
355,ref_0_que_1-7,sqeuclidean,0,0.816645,0.811872,0.799670,0.805725,0.901449,0.901360,0.034201,0.468322,34.998755
356,ref_0_que_1-7,sqeuclidean,100,0.815425,0.802904,0.810850,0.806858,0.901479,0.902908,0.036622,0.480174,35.062088
357,ref_0_que_1-7,sqeuclidean,200,0.816732,0.807220,0.807368,0.807294,0.900762,0.902468,0.031477,0.475556,34.570408
358,ref_0_que_1-7,sqeuclidean,300,0.818475,0.808600,0.809934,0.809267,0.902028,0.903367,0.032990,0.476253,35.205270


In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load your DataFrame (assuming it's called df)
# df = pd.read_csv("your_data.csv")  # or already in memory

# Melt the metric columns into long format
metrics = ['accuracy', 'precision', 'recall', 'f1', 'auc', 'aupr']
df_long = df.melt(id_vars=['PROBLEM_KEY', 'dist', 'seed'], value_vars=metrics,
                  var_name='metric', value_name='value')


In [15]:
import pandas as pd

# Group by distance and PROBLEM_KEY to compute mean AUC
auc_table = df.groupby(['dist', 'PROBLEM_KEY'])['auc'].mean().unstack()

# Rank each column (PROBLEM_KEY), higher AUC = better rank (descending)
ranked = auc_table.rank(ascending=False)

# Compute mean rank for each distance metric
mean_ranks = ranked.mean(axis=1)

# Add mean_rank column to the AUC table
auc_table['mean_rank'] = mean_ranks

# Sort the AUC table by mean_rank (lower rank = better)
auc_table_sorted = auc_table.sort_values('mean_rank')

# Optionally round for clarity
auc_table_sorted = auc_table_sorted.round(3)

auc_table_sorted.to_csv("auc_metrics.csv")

auc_table_sorted

PROBLEM_KEY,ref_0_que_1,ref_0_que_1-7,ref_0_que_1hr,ref_0_que_3,ref_0_que_4hr,ref_0_que_7,mean_rank
dist,,,,,,,
chebyshev,0.951,0.902,0.921,0.933,0.947,0.843,2.333
euclidean,0.951,0.901,0.917,0.932,0.940,0.850,4.167
mse,0.951,0.901,0.917,0.932,0.940,0.850,4.167
minkowski,0.951,0.901,0.917,0.932,0.940,0.850,4.167
sqeuclidean,0.951,0.901,0.917,0.932,0.940,0.850,4.167
mae,0.950,0.901,0.913,0.932,0.926,0.853,6.333
cityblock,0.950,0.901,0.913,0.932,0.926,0.853,6.333
mahalanobis,0.948,0.877,0.915,0.928,0.947,0.851,7.000
seuclidean,0.948,0.878,0.915,0.931,0.947,0.836,7.000


In [16]:
import pandas as pd

# Group by distance and PROBLEM_KEY to compute mean AUC
aupr_table = df.groupby(['dist', 'PROBLEM_KEY'])['aupr'].mean().unstack()

# Rank each column (PROBLEM_KEY), higher AUC = better rank (descending)
ranked = aupr_table.rank(ascending=False)

# Compute mean rank for each distance metric
mean_ranks = ranked.mean(axis=1)

# Add mean_rank column to the AUC table
aupr_table['mean_rank'] = mean_ranks

# Sort the AUC table by mean_rank (lower rank = better)
aupr_table_sorted = aupr_table.sort_values('mean_rank')

# Optionally round for clarity
aupr_table_sorted = aupr_table_sorted.round(3)

aupr_table_sorted.to_csv("aupr_metrics.csv")


aupr_table_sorted

PROBLEM_KEY,ref_0_que_1,ref_0_que_1-7,ref_0_que_1hr,ref_0_que_3,ref_0_que_4hr,ref_0_que_7,mean_rank
dist,,,,,,,
chebyshev,0.959,0.903,0.859,0.897,0.963,0.811,2.167
euclidean,0.959,0.902,0.841,0.895,0.958,0.822,4.000
mse,0.959,0.902,0.841,0.895,0.958,0.822,4.000
minkowski,0.959,0.902,0.841,0.895,0.958,0.822,4.000
sqeuclidean,0.959,0.902,0.841,0.895,0.958,0.822,4.000
mae,0.958,0.902,0.821,0.894,0.946,0.824,6.333
cityblock,0.958,0.902,0.821,0.894,0.946,0.824,6.333
seuclidean,0.957,0.887,0.839,0.893,0.961,0.791,6.833
mahalanobis,0.957,0.886,0.837,0.888,0.960,0.815,7.333
